# Machine Learning: Validação de Modelos

* **Base de Dados:** Características físicas de alguns carros que estão ou estiveram à venda, seus preços de venda, e se foram vendidos ou não. Site de vendas fictício.

* **Sobre o projeto:** Este projeto está sendo desenvolvido com referência no curso [Machine Learning: validação de modelos](https://cursos.alura.com.br/course/machine-learning-validando-modelos) da Alura, que é a continuação direta de um outro curso [Machine Learning: classificação com SKLearn](https://cursos.alura.com.br/course/machine-learning-introducao-a-classificacao-com-sklearn). Por isso, a inicialização deste projeto não terá muitas observações.

## Abertura e Tratamentos dos dados

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   preco            10000 non-null  float64
 1   vendido          10000 non-null  int64  
 2   idade_do_modelo  10000 non-null  int64  
 3   km_por_ano       10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print(f"Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos")

Treinaremos com 7500 elementos e testaremos com 2500 elementos


## Modelo baseline (Dummy Classifier)

* A taxa de acerto do modelo treinado com Dummy Classifier será utilizada como referência para comparações entre outros modelos.

In [9]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print(f"A acurácia do Dummy Stratified foi de {acuracia:.2f}%")

A acurácia do Dummy Stratified foi de 48.96%


## Modelo 1: Decision Tree

* Profundidade máxima 2.

In [11]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print (f"A acurácia foi {acuracia:.2f}%")

A acurácia foi 71.92%


## Efeitos da aleatoriedade

* A SEED escolhida nos algoritmos com fatores aleatórios pode mudar muito a taxa de acerto dos modelos, e pode causar decisões equivocadas em decisões de negócio.

In [12]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print (f"A acurácia foi {acuracia:.2f}%")

A acurácia foi 76.84%


* O modelo pode ser rodado diversas vezes até que uma estimativa da acurácia seja calculada, com uma margem de acerto, não apenas uma porcentagem fixa altamente relacionada ao fato aleatório.

# Validação cruzada (Cross Validation)

* Variações nos conjuntos de treino e teste dos modelos.

* k-fold e cross validation.

* O k é determinante na confiabilidade da acurácia do modelo. No geral, quanto mais divisões, mais vezes o modelos será avaliado e mais precisa será a acurácia média. Entretanto, quando mais modelos são treinados, mais cara fica a validação.